In [ ]:
!git clone --quiet https://github.com/cvg/Hierarchical-Localization
%cd Hierarchical-Localization
!git submodule update --init --recursive
!pip install --progress-bar off --quiet -e .
%cd ..

!git clone --quiet https://github.com/cvg/LightGlue/
%cd LightGlue
!pip install --progress-bar off --quiet -e .
%cd ..

!pip install --progress-bar off --quiet pycolmap

!sudo apt-get update -qq
!sudo apt-get install -y unrar p7zip-full

from google.colab import drive
drive.mount('/content/drive')

/content/Hierarchical-Localization
Submodule 'third_party/SuperGluePretrainedNetwork' (https://github.com/magicleap/SuperGluePretrainedNetwork.git) registered for path 'third_party/SuperGluePretrainedNetwork'
Submodule 'third_party/d2net' (https://github.com/mihaidusmanu/d2-net.git) registered for path 'third_party/d2net'
Submodule 'third_party/deep-image-retrieval' (https://github.com/naver/deep-image-retrieval.git) registered for path 'third_party/deep-image-retrieval'
Submodule 'third_party/r2d2' (https://github.com/naver/r2d2.git) registered for path 'third_party/r2d2'
Cloning into '/content/Hierarchical-Localization/third_party/SuperGluePretrainedNetwork'...
Cloning into '/content/Hierarchical-Localization/third_party/d2net'...
Cloning into '/content/Hierarchical-Localization/third_party/deep-image-retrieval'...
Cloning into '/content/Hierarchical-Localization/third_party/r2d2'...
Submodule path 'third_party/SuperGluePretrainedNetwork': checked out 'ddcf11f42e7e0732a0c4607648f9448

In [ ]:
import subprocess
from pathlib import Path
import os

source_file_path_str = '/content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip'
source_file = Path(source_file_path_str)
local_data_dir = Path('/content/images_dataset')
local_data_dir.mkdir(exist_ok=True)
command = []
suffix = source_file.suffix.lower()

if not source_file.exists():
    print(f"Error: No source file: {source_file}")
else:
    print(f"Source file: {source_file.name}")
    if suffix == '.zip':
        command = ['unzip', '-o', str(source_file), '-d', str(local_data_dir)]
    else:
        print(f"Error: Script using .zip")

    if command:
        print(f"{' '.join(command)}")
        result = subprocess.run(command, capture_output=True, text=True)
        if result.returncode == 0:
            print("Success")
            !ls -l {local_data_dir} | head -n 10
        else:
            print(f"Error")
            print(result.stderr)

Đã tìm thấy file: FrameBuffer_RGB.zip
Đang thực thi: unzip -o /content/drive/MyDrive/3D_reconstruction/datasets/FrameBuffer_RGB.zip -d /content/images_dataset
--- Giải nén thành công! ---
total 56
drwxr-xr-x 2 root root 53248 Jan  6 23:23 FrameBuffer_RGB


In [ ]:
import sys
from pathlib import Path
from datetime import datetime

sys.path.append('/content/Hierarchical-Localization')
sys.path.append('/content/LightGlue')

dataset_name = 'FrameBuffer_RGB'
base_dir = Path('/content/images_dataset/')
image_dir = base_dir

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f'{dataset_name}_PINHOLE_{timestamp}'

outputs = Path(f'/content/outputs/{run_name}')
outputs.mkdir(exist_ok=True, parents=True)

sfm_dir = outputs / 'sfm'
features_global = outputs / 'global_features.h5'
features_local = outputs / 'local_features.h5'
matches = outputs / 'matches.h5'
sfm_pairs = outputs / 'pairs-sequential.txt'

print(f"Dataset: {dataset_name}")
print(f"Output dir: {outputs}")

Dataset: FrameBuffer_RGB
Output dir: /content/outputs/FrameBuffer_RGB_PINHOLE_20260126_193017


In [ ]:
from hloc import extract_features, match_features, reconstruction
from hloc.utils.io import list_h5_names
from pathlib import Path

feature_conf = {
    'output': 'feats-superpoint',
    'model': {
        'name': 'superpoint',
        'model_path': '/content/drive/MyDrive/3D_reconstruction/models/SuperpointCE.pth',
        'max_keypoints': 20480,
        'keypoint_threshold': 0.01,
        'nms_radius': 4,
    },
    'preprocessing': {
        'grayscale': True,
    }
}

matcher_conf = {
    'output': 'matches-lightglue',
    'model': {
        'name': 'lightglue',
        'features': 'superpoint',
        'filter_threshold': 0.1,
        'depth_confidence': 0.95,
        'width_confidence': 0.99,
    },
}

In [ ]:
import pycolmap
from hloc import extract_features, match_features, reconstruction
from hloc.utils.io import list_h5_names
from pathlib import Path

def patched_incremental_mapping(database_path, image_dir, sfm_path, options=None):
    if options is None: options = {}
    pipeline_options = pycolmap.IncrementalPipelineOptions()
    for key, value in options.items():
        if hasattr(pipeline_options, key): setattr(pipeline_options, key, value)
        elif hasattr(pipeline_options.mapper, key): setattr(pipeline_options.mapper, key, value)
    return pycolmap.incremental_mapping(database_path, image_dir, sfm_path, options=pipeline_options)
reconstruction.incremental_mapping = patched_incremental_mapping

def generate_sequential_pairs(features_path, output_pairs_path, window_size=20):
    names = list_h5_names(features_path)
    names = sorted(names)
    n = len(names)

    pairs = set()
    for i in range(n):
        for offset in range(1, window_size + 1):
            if i + offset < n:
                pairs.add((names[i], names[i + offset]))
            if i - offset >= 0:
                pair = tuple(sorted([names[i], names[i - offset]]))
                pairs.add(pair)

    unique_pairs = []
    for p in pairs:
        sorted_pair = tuple(sorted(p))
        if sorted_pair not in unique_pairs:
            unique_pairs.append(sorted_pair)

    print(f"Total images: {n}")
    print(f"Pairs: (window={window_size}): {len(unique_pairs)}")

    with open(output_pairs_path, "w") as f:
        f.write("\n".join(" ".join([p[0], p[1]]) for p in unique_pairs))

    return unique_pairs

# PIPELINE
print("Extracting features")
extract_features.main(feature_conf, image_dir, feature_path=features_local)

print("\n Generating sequential pairs")
generate_sequential_pairs(features_local, sfm_pairs, window_size=20)

print("\n Matching features")
match_features.main(matcher_conf, sfm_pairs, features=features_local, matches=matches)

# PINHOLE model format: fx, fy, cx, cy
camera_params_str = "227.60416, 237.5, 227.60416, 237.5"

mapper_options = {
    "init_min_tri_angle": 2.0,
    "init_min_num_inliers": 15,

    "abs_pose_min_num_inliers": 12,
    "abs_pose_max_error": 6.0, #
    "filter_max_reproj_error": 4.0,

    # --- 3. BUNDLE ADJUSTMENT ---
    "ba_refine_focal_length": False,
    "ba_refine_principal_point": False,
    "ba_refine_extra_params": False,

    "ba_global_images_ratio": 1.1,
    "ba_global_points_ratio": 1.1,
    "ba_global_max_num_iterations": 20,
}

image_options = {
    "camera_model": "PINHOLE",
    "camera_params": camera_params_str
}

print(f"\n SfM")
reconstruction.main(
    sfm_dir, image_dir, sfm_pairs, features_local, matches,
    mapper_options=mapper_options,
    image_options=image_options,
    camera_mode=pycolmap.CameraMode.SINGLE,
)

[2026/01/26 19:30:24 hloc INFO] Extracting local features with configuration:
{'model': {'keypoint_threshold': 0.01,
           'max_keypoints': 20480,
           'model_path': '/content/drive/MyDrive/3D_reconstruction/models/SuperpointCE.pth',
           'name': 'superpoint',
           'nms_radius': 4},
 'output': 'feats-superpoint',
 'preprocessing': {'grayscale': True}}
[2026/01/26 19:30:24 hloc INFO] Found 1201 images in root /content/images_dataset.


[1/3] Extracting features...
Loaded SuperPoint model


100%|██████████| 1201/1201 [00:20<00:00, 57.36it/s]
[2026/01/26 19:30:46 hloc INFO] Finished exporting features.



[2/3] Generating sequential pairs...


[2026/01/26 19:30:55 hloc INFO] Matching local features with configuration:
{'model': {'depth_confidence': 0.95,
           'features': 'superpoint',
           'filter_threshold': 0.1,
           'name': 'lightglue',
           'width_confidence': 0.99},
 'output': 'matches-lightglue'}


Tổng số ảnh: 1201
Số cặp sequential (window=20): 23810
So với exhaustive: 720600 cặp

[3/3] Matching features...
Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_lightglue.pth" to /root/.cache/torch/hub/checkpoints/superpoint_lightglue_v0-1_arxiv.pth


100%|██████████| 45.3M/45.3M [00:00<00:00, 101MB/s] 
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100%|██████████| 23810/23810 [13:29<00:00, 29.43it/s]
[2026/01/26 19:44:31 hloc INFO] Finished exporting matches.
[2026/01/26 19:44:31 hloc INFO] Writing COLMAP logs to /content/outputs/FrameBuffer_RGB_PINHOLE_20260126_193017/sfm/colmap.LOG.*
[2026/01/26 19:44:31 hloc INFO] Creating an empty database...
[2026/01/26 19:44:31 hloc INFO] Importing images into the database...



--- Bắt đầu SfM với PINHOLE model ---
Camera params: 227.60416, 237.5, 227.60416, 237.5


[2026/01/26 19:44:39 hloc INFO] Importing features into the database...
100%|██████████| 1201/1201 [00:00<00:00, 1278.78it/s]
[2026/01/26 19:44:40 hloc INFO] Importing matches into the database...
100%|██████████| 23810/23810 [00:26<00:00, 910.32it/s]
[2026/01/26 19:45:06 hloc INFO] Performing geometric verification of the matches...
[2026/01/26 19:47:11 hloc INFO] Running 3D reconstruction...
[2026/01/26 19:59:19 hloc INFO] Reconstructed 1 model(s).
[2026/01/26 19:59:19 hloc INFO] Largest model is #0 with 1201 images.
[2026/01/26 19:59:19 hloc INFO] Reconstruction statistics:
Reconstruction:
	num_rigs = 1
	num_cameras = 1
	num_frames = 1201
	num_reg_frames = 1201
	num_images = 1201
	num_points3D = 18808
	num_observations = 187568
	mean_track_length = 9.97278
	mean_observations_per_image = 156.177
	mean_reprojection_error = 0.882611
	num_input_images = 1201


Reconstruction(num_rigs=1, num_cameras=1, num_frames=1201, num_reg_frames=1201, num_images=1201, num_points3D=18808)

In [ ]:
import pycolmap
from pathlib import Path
import os

print(f"Đang tìm model tại: {sfm_dir}")

if not sfm_dir.exists() or not any(sfm_dir.iterdir()):
    print("Error: No SfM file")
else:
    if (sfm_dir / "0").exists():
        model_path = sfm_dir / "0"
    else:
        model_path = sfm_dir

    print(f"Loading model từ: {model_path}")

    try:
        reconstruction = pycolmap.Reconstruction(model_path)
        output_ply = outputs / 'sparse_model.ply'
        reconstruction.export_PLY(str(output_ply))
        print(f"PLY output: {output_ply}")
    except ValueError as e:
        print(f"PyCOLMAP eror: {e}")

Đang tìm model tại: /content/outputs/FrameBuffer_RGB_PINHOLE_20260126_193017/sfm
Loading model từ: /content/outputs/FrameBuffer_RGB_PINHOLE_20260126_193017/sfm
✅ Đã xuất file PLY thành công tại: /content/outputs/FrameBuffer_RGB_PINHOLE_20260126_193017/sparse_model.ply

📊 Thống kê model:
   - Số cameras: 1
   - Số images registered: 1201
   - Số points 3D: 18808


In [ ]:
# VISUALIZE 3D
from hloc.utils.viz_3d import init_figure, plot_reconstruction
import pycolmap

fig = init_figure()
plot_reconstruction(
    fig,
    reconstruction,
    color='rgba(255,0,0,0.5)',
    name='sparse_model_viz',
    points_rgb=False
)

fig.show()

In [ ]:
import shutil
import os
from pathlib import Path

outputs_root = Path('/content/outputs')
drive_dest = Path('/content/drive/MyDrive/3D_reconstruction/outputs')
drive_dest.mkdir(exist_ok=True, parents=True)

latest_run = max(outputs_root.iterdir(), key=os.path.getmtime)
sfm_output_dir = latest_run / 'sfm'

zip_filename = f"{latest_run.name}_results"
shutil.make_archive(f"/content/{zip_filename}", 'zip', latest_run)
shutil.copy2(f"/content/{zip_filename}.zip", drive_dest / f"{zip_filename}.zip")

print(f"Save: {drive_dest}/{zip_filename}.zip")

💾 Đang xử lý kết quả từ: FrameBuffer_RGB_PINHOLE_20260126_193017
✅ Đã lưu xong: /content/drive/MyDrive/3D_reconstruction/outputs/FrameBuffer_RGB_PINHOLE_20260126_193017_results.zip
